[View in Colaboratory](https://colab.research.google.com/github/Xueping/colab_ml/blob/master/py2neo_creates_patient_journey.ipynb)

In [0]:
from py2neo import Graph, Node, Relationship,authenticate
import numpy as np
import pandas as pd

In [0]:
# set up authentication parameters
authenticate("localhost:7474", "neo4j", "admin")

# connect to authenticated graph database
graph = Graph("http://localhost:7474/db/data/")
# grapp = Graph()

In [0]:
patients = pd.read_csv('../doh/mimic3/data/PATIENTS.csv')
admissions = pd.read_csv('../doh/mimic3/data/ADMISSIONS.csv')

In [0]:
unique_pats = admissions.SUBJECT_ID.unique()
profs = pd.merge(admissions,patients,how='left',on='SUBJECT_ID').\
            drop(['ROW_ID_y','ROW_ID_x','DOB','DOD','DOD_HOSP','DOD_SSN','EXPIRE_FLAG'], axis=1)

In [0]:
profs.shape,  admissions.shape,unique_pats.shape

((58976, 19), (58976, 19), (46520,))

In [0]:
grouped = profs.groupby(['SUBJECT_ID'])
samples = unique_pats[range(10000)]

In [0]:
tx = graph.begin()
# for sub_id in unique_pats[0:1000]:
for sub_id in samples:
    pat_acts = grouped.get_group(sub_id).sort_values(by=['ADMITTIME'])
    last_act = pat_acts.tail(1)
    p = Node("Patient", \
             pid=str(last_act.SUBJECT_ID.values[0]),\
             insurance=last_act.INSURANCE.values[0],\
             religion=last_act.RELIGION.values[0],\
             marital=last_act.MARITAL_STATUS.values[0],\
             ethnicity=last_act.ETHNICITY.values[0],\
             gender=last_act.GENDER.values[0])
    tx.create(p)
    
    for index, row in pat_acts.iterrows():
        act = Node("Activity", \
                   hadm=row['HADM_ID'],\
                   admit_time=row['ADMITTIME'],\
                   disch_time=row['DISCHTIME'],\
                   admit_type=row['ADMISSION_TYPE'],\
                   admit_loc=row['ADMISSION_LOCATION'],\
                   disch_loc=row['DISCHARGE_LOCATION'],\
                   diag=row['DIAGNOSIS'])
        tx.create(act)
        hsp = Relationship(p, "Hospital", act)
        tx.create(hsp)
        p = act
tx.commit()   


In [0]:
df = pd.DataFrame(samples,columns=['pid'])

In [0]:
#path = pd.DataFrame(graph.data("MATCH (n:Patient)-[:Hospital]->(act1)-[:Hospital]->(act) RETURN n.pid as pid, 1 as flag"))

In [0]:
#path.pid = path.pid.astype(int)

In [0]:
#df = pd.merge(newData,path,on='pid',how='left')

In [0]:
adm_type = ['EMERGENCY','ELECTIVE','NEWBORN','URGENT']
paths = []
for l in range(1,5): 
    for adm in adm_type:
        p = "MATCH (n:Patient)-[:Hospital*"+str(l)+"]->(act) WHERE act.admit_type='"\
                + adm +"' RETURN n.pid as pid, 1 as path_"+str(l)+"_"+adm
        paths.append(p)

In [0]:
paths

["MATCH (n:Patient)-[:Hospital*1]->(act) WHERE act.admit_type='EMERGENCY' RETURN n.pid as pid, 1 as path_1_EMERGENCY",
 "MATCH (n:Patient)-[:Hospital*1]->(act) WHERE act.admit_type='ELECTIVE' RETURN n.pid as pid, 1 as path_1_ELECTIVE",
 "MATCH (n:Patient)-[:Hospital*1]->(act) WHERE act.admit_type='NEWBORN' RETURN n.pid as pid, 1 as path_1_NEWBORN",
 "MATCH (n:Patient)-[:Hospital*1]->(act) WHERE act.admit_type='URGENT' RETURN n.pid as pid, 1 as path_1_URGENT",
 "MATCH (n:Patient)-[:Hospital*2]->(act) WHERE act.admit_type='EMERGENCY' RETURN n.pid as pid, 1 as path_2_EMERGENCY",
 "MATCH (n:Patient)-[:Hospital*2]->(act) WHERE act.admit_type='ELECTIVE' RETURN n.pid as pid, 1 as path_2_ELECTIVE",
 "MATCH (n:Patient)-[:Hospital*2]->(act) WHERE act.admit_type='NEWBORN' RETURN n.pid as pid, 1 as path_2_NEWBORN",
 "MATCH (n:Patient)-[:Hospital*2]->(act) WHERE act.admit_type='URGENT' RETURN n.pid as pid, 1 as path_2_URGENT",
 "MATCH (n:Patient)-[:Hospital*3]->(act) WHERE act.admit_type='EMERGENCY

In [0]:
for p in paths:
    graphData = graph.data(p)
    if(len(graphData)>0):
        p_df = pd.DataFrame(graphData)
        p_df.pid = p_df.pid.astype(int)
        df = pd.merge(df,p_df,on='pid',how='left')

In [0]:
df = df.fillna(0)

In [0]:
df

,pid,path_1_EMERGENCY,path_1_ELECTIVE,path_1_NEWBORN,path_1_URGENT,path_2_EMERGENCY,path_2_ELECTIVE,path_2_NEWBORN,path_2_URGENT,path_3_EMERGENCY,path_3_ELECTIVE,path_3_NEWBORN,path_3_URGENT,path_4_EMERGENCY,path_4_ELECTIVE,path_4_URGENT
0,22,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,24,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,25,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,27,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,28,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,30,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,31,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,32,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
